In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output, display
from io import StringIO

In [ ]:
CHARLESTON_2024="https://www.runrevel.com/rmc/results?sort=&race=166959&date=&event=Marathon&gender=&division=&search=&loc_167831=&page_167831={}&size_167831=1000"
CHARLESTON_2023="https://www.runrevel.com/rmc/results?sort=&race=166804&date=&event=Marathon&gender=&division=&search=&loc_167206=&page_167206={}&size_167206=1000"
BIG_BEAR_2022 = "https://www.runrevel.com/rbb/results?sort=&race=166771&date=&event=Marathon&gender=&division=&search=&loc_167125=&page_167125={}&size_167125=1000"

In [ ]:
URL = BIG_BEAR_2022
YEAR = 2022
NAME = "bigbear_" + str(YEAR)
PAGE = 167125


In [ ]:
df_final = None

page_number = 1

while True:
    clear_output()
    display(f"Retrieving page {page_number}")


    res = requests.get(URL.format(page_number))
    page = BeautifulSoup(res.text)
    pager = page.find('select', id=f'page_{PAGE}')
    tbl = pager.parent.parent.find_next_sibling('table', id="roster")
    if tbl is None:
        break
    df=pd.read_html(StringIO(str(tbl)))[0]

    if df_final is None:
        df_final = df
    else:
        df_final = pd.concat([df_final, df])

    page_number += 1

    pages = pager.find_all('option')
    if page_number> len(pages):
        break

df_final.shape

In [ ]:
df_final.reset_index(drop=True, inplace=True)

In [ ]:

df_final.to_csv(NAME + '_full.csv')
df_final[['Gender','Age','Chip Time']].to_csv(NAME + '.csv')

In [ ]:
df_final.pivot_table(index='Age', columns='Gender', aggfunc='size', fill_value=0)